In [2]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import pandas as pd
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [2]:
state_links=["https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
             "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"]

**KERALA BUS ROUTES AND LINKS**

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# Open the desired webpage
driver.get("https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def kerala_link_route(path):
    LINKS_KERALA = []
    ROUTE_KERALA = []
    
    # WebDriverWait instance
    wait = WebDriverWait(driver, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver.find_elements(By.XPATH, path)   #here path variable is used for getting arguments and the parameter is fixed ...it returns the element which contains the text and link
            for links in paths:  #one page contains many anchor tags with class = "routes" from that we traverse using 'links' variable 
                d = links.get_attribute("href")  #get_attribute is used to get the value used inside the html attribute...it is where we have retrieved link from .
                if d:
                    LINKS_KERALA.append(d)  #d is the link now...we have appended/added in the LINKS_KERALA list
            
            for route in paths:
                ROUTE_KERALA.append(route.text)  #similarly for each anchor tag class='route', we are retrieving the route name by using route.text where .text is used to get value from the element
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']") 
                active_page_number = active_page_element.text #the pagination has numbers in it...that number is stored by using .text in active_page_number 
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"  #text() function is used to match the content with which we provided in the descendent of the div[class = " DC_117_pagination"]
                next_page_button = wait.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))  #we have to wait untill the webdriver loads the nextpage's XPATH
                
                # Scroll into view and click using JavaScript if necessary
                driver.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click() #we waited until next page xpath loaded...now after loaded we click
                except ElementNotInteractableException:
                    driver.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content  #this runs in loop...each page loads its data in 'try'  and then after finishing another page loads
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found") 
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_KERALA, ROUTE_KERALA  

# Retrieve links and routes
LINKS_KERALA, ROUTE_KERALA = kerala_link_route("//a[@class='route']")  #function calling and argument is passed as XPATH 

# Close the WebDriver
driver.quit()

# Save data to a CSV file
df_k = pd.DataFrame({"Route_name": ROUTE_KERALA, "Route_link": LINKS_KERALA})

df_k.to_csv("dummy.csv", index=False)


Navigating to page 2
No more pages to paginate or pagination element not found


In [ ]:
# path=r"E:/Guvi DS/RedbusScrapingProject/df_k.csv"
# df_k.to_csv(path,index=False)

In [4]:
df_k

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
5,Kozhikode to Mysore,https://www.redbus.in/bus-tickets/kozhikode-to...
6,Kozhikode to Thrissur,https://www.redbus.in/bus-tickets/kozhikode-to...
7,Kozhikode to Thiruvananthapuram,https://www.redbus.in/bus-tickets/kozhikode-to...
8,Kozhikode to Kottayam,https://www.redbus.in/bus-tickets/kozhikode-to...
9,Thrissur to Kozhikode,https://www.redbus.in/bus-tickets/thrissur-to-...


ANDHRA BUS ROUTES AND LINKS

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_A = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_A.maximize_window()

# Open the desired webpage
driver_A.get("https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def andhra_link_route(path):
    LINKS_ANDHRA = []
    ROUTE_ANDHRA = []
    
    # WebDriverWait instance
    wait_A = WebDriverWait(driver_A, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_A.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_ANDHRA.append(d)
            
            for route in paths:
                ROUTE_ANDHRA.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_A.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_A.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_A.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_A.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_ANDHRA, ROUTE_ANDHRA

# Retrieve links and routes
LINKS_ANDHRA, ROUTE_ANDHRA = andhra_link_route("//a[@class='route']")

# Close the WebDriver
driver_A.quit()

# Save data to a CSV file
df_a = pd.DataFrame({"Route_name": ROUTE_ANDHRA, "Route_link": LINKS_ANDHRA})
df_a.to_csv("df_a.csv", index=False)


Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [8]:
df_a

,Route_name,Route_link
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
1,Vijayawada to Hyderabad,https://www.redbus.in/bus-tickets/vijayawada-t...
2,Kakinada to Visakhapatnam,https://www.redbus.in/bus-tickets/kakinada-to-...
3,Chittoor (Andhra Pradesh) to Bangalore,https://www.redbus.in/bus-tickets/chittoor-and...
4,Bangalore to Tirupati,https://www.redbus.in/bus-tickets/bangalore-to...
5,Tirupati to Bangalore,https://www.redbus.in/bus-tickets/tirupathi-to...
6,Kadapa to Bangalore,https://www.redbus.in/bus-tickets/kadapa-to-ba...
7,Hyderabad to Ongole,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Anantapur (andhra pradesh) to Bangalore,https://www.redbus.in/bus-tickets/ananthapur-t...
9,Bangalore to Kadapa,https://www.redbus.in/bus-tickets/bangalore-to...


TELUNGANA BUS ROUTES AND LINKS

In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_T = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_T.maximize_window()

# Open the desired webpage
driver_T.get("https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def telungana_link_route(path):
    LINKS_TELUNGANA = []
    ROUTE_TELUNGANA = []
    
    # WebDriverWait instance
    wait_T = WebDriverWait(driver_T, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_T.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_TELUNGANA.append(d)
            
            for route in paths:
                ROUTE_TELUNGANA.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_T.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_T.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_T.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_T.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_TELUNGANA, ROUTE_TELUNGANA

# Retrieve links and routes
LINKS_TELUNGANA, ROUTE_TELUNGANA = telungana_link_route("//a[@class='route']")

# Close the WebDriver
driver_T.quit()

# Save data to a CSV file
df_t = pd.DataFrame({"Route_name": ROUTE_TELUNGANA, "Route_link": LINKS_TELUNGANA})
df_t.to_csv("df_t.csv", index=False)


Navigating to page 2
Navigating to page 3
No more pages to paginate or pagination element not found


In [10]:
df_t

,Route_name,Route_link
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Kothagudem to Hyderabad,https://www.redbus.in/bus-tickets/kothagudem-t...
7,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Guntur (Andhra Pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/guntur-to-hy...
9,Godavarikhani to Hyderabad,https://www.redbus.in/bus-tickets/godavarikhan...


GOA BUS ROUTES AND LINKS

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_G = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_G.maximize_window()

# Open the desired webpage
driver_G.get("https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def goa_link_route(path):
    LINKS_GOA = []
    ROUTE_GOA = []
    
    # WebDriverWait instance
    wait_G = WebDriverWait(driver_G, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_G.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_GOA.append(d)
            
            for route in paths:
                ROUTE_GOA.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_G.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_G.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_G.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_G.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_GOA, ROUTE_GOA

# Retrieve links and routes
LINKS_GOA, ROUTE_GOA = goa_link_route("//a[@class='route']")

# Close the WebDriver
driver_G.quit()

# Save data to a CSV file
df_g = pd.DataFrame({"Route_name": ROUTE_TELUNGANA, "Route_link": LINKS_TELUNGANA})
df_g.to_csv("df_g.csv", index=False)


Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [12]:
df_g

,Route_name,Route_link
0,Khammam to Hyderabad,https://www.redbus.in/bus-tickets/khammam-to-h...
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...
2,Hyderabad to Khammam,https://www.redbus.in/bus-tickets/hyderabad-to...
3,Hyderabad to Srisailam,https://www.redbus.in/bus-tickets/hyderabad-to...
4,Karimnagar to Hyderabad,https://www.redbus.in/bus-tickets/karimnagar-t...
5,Hyderabad to Adilabad,https://www.redbus.in/bus-tickets/hyderabad-to...
6,Kothagudem to Hyderabad,https://www.redbus.in/bus-tickets/kothagudem-t...
7,Hyderabad to Mancherial,https://www.redbus.in/bus-tickets/hyderabad-to...
8,Guntur (Andhra Pradesh) to Hyderabad,https://www.redbus.in/bus-tickets/guntur-to-hy...
9,Godavarikhani to Hyderabad,https://www.redbus.in/bus-tickets/godavarikhan...


RAJASTHAN BUS ROUTES AND LINKS


In [13]:
driver_R = webdriver.Chrome()

driver_R.get("https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile")
driver_R.maximize_window()
time.sleep(3)





# Function to retrieve bus links and routes
def rajasthan_link_route(path):
    LINKS_RAJASTHAN = []
    ROUTE_RAJASTHAN = []
    
    # WebDriverWait instance
    wait_R = WebDriverWait(driver_R, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_R.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_RAJASTHAN.append(d)
            
            for route in paths:
                ROUTE_RAJASTHAN.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_R.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_R.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_R.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_R.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_RAJASTHAN, ROUTE_RAJASTHAN

# Retrieve links and routes
LINKS_RAJASTHAN, ROUTE_RAJASTHAN = rajasthan_link_route("//a[@class='route']")

# Close the WebDriver
driver_R.quit()

# Save data to a CSV file
df_r = pd.DataFrame({"Route_name": ROUTE_RAJASTHAN, "Route_link": LINKS_RAJASTHAN})
df_r.to_csv("df_r.csv", index=False)


Navigating to page 2
No more pages to paginate or pagination element not found


In [14]:
df_r

,Route_name,Route_link
0,Jodhpur to Ajmer,https://www.redbus.in/bus-tickets/jodhpur-to-a...
1,Beawar (Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/beawer-to-ja...
2,Udaipur to Jodhpur,https://www.redbus.in/bus-tickets/udaipur-to-j...
3,Jaipur (Rajasthan) to Jodhpur,https://www.redbus.in/bus-tickets/jaipur-to-jo...
4,Sikar to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/sikar-to-jaipur
5,Aligarh (uttar pradesh) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/aligarh-utta...
6,Kota(Rajasthan) to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kota-rajasth...
7,Jaipur (Rajasthan) to Pilani,https://www.redbus.in/bus-tickets/jaipur-to-pi...
8,Jaipur (Rajasthan) to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/jaipur-to-al...
9,Kishangarh to Jaipur (Rajasthan),https://www.redbus.in/bus-tickets/kishangarh-t...


SOUTH BENGAL BUS ROUTES AND LINKS

In [15]:
driver_S = webdriver.Chrome()

driver_S.get("https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile")
driver_S.maximize_window()

time.sleep(10)



# Function to retrieve bus links and routes
def southbengal_link_route(path):
    LINKS_SB = []
    ROUTE_SB = []
    
    # WebDriverWait instance
    wait_S = WebDriverWait(driver_S, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_S.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_SB.append(d)
            
            for route in paths:
                ROUTE_SB.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_S.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_S.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_S.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_S.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_SB, ROUTE_SB

# Retrieve links and routes
LINKS_SB, ROUTE_SB = southbengal_link_route("//a[@class='route']")

# Close the WebDriver
driver_S.quit()

# Save data to a CSV file
df_s = pd.DataFrame({"Route_name": ROUTE_SB, "Route_link": LINKS_SB})
df_s.to_csv("df_s.csv", index=False)


Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [16]:
df_s

,Route_name,Route_link
0,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
1,Kolkata to Burdwan,https://www.redbus.in/bus-tickets/kolkata-to-b...
2,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
3,Haldia to Kolkata,https://www.redbus.in/bus-tickets/haldia-to-ko...
4,Kolkata to Haldia,https://www.redbus.in/bus-tickets/kolkata-to-h...
5,Midnapore to Kolkata,https://www.redbus.in/bus-tickets/midnapore-to...
6,Kolkata to Arambagh (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-a...
7,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
8,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
9,Kolkata to Bankura,https://www.redbus.in/bus-tickets/kolkata-to-b...


HARYANA BUS ROUTES AND LINKS

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_H = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_H.maximize_window()

# Open the desired webpage
driver_H.get("https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def haryana_link_route(path):
    LINKS_HY = []
    ROUTE_HY = []
    
    # WebDriverWait instance
    wait_H = WebDriverWait(driver_H, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_H.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_HY.append(d)
            
            for route in paths:
                ROUTE_HY.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_H.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_H.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_H.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_H.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_HY, ROUTE_HY

# Retrieve links and routes
LINKS_HY, ROUTE_HY = haryana_link_route("//a[@class='route']")

# Close the WebDriver
driver_H.quit()

# Save data to a CSV file
df_h = pd.DataFrame({"Route_name": ROUTE_HY, "Route_link": LINKS_HY})
df_h.to_csv("df_h.csv", index=False)


Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found


In [18]:
df_h

,Route_name,Route_link
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla
1,Hamirpur (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/hamirpur-him...
2,Chandigarh to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/chandigarh-t...
3,Shimla to Delhi,https://www.redbus.in/bus-tickets/shimla-to-delhi
4,Hamirpur (Himachal Pradesh) to Delhi,https://www.redbus.in/bus-tickets/hamirpur-him...
5,Delhi to Hamirpur (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-ham...
6,Kangra to Chandigarh,https://www.redbus.in/bus-tickets/kangra-to-ch...
7,Chamba (Himachal Pradesh) to Chandigarh,https://www.redbus.in/bus-tickets/chamba-himac...
8,Delhi to Chandigarh,https://www.redbus.in/bus-tickets/delhi-to-cha...
9,Palampur to Chandigarh,https://www.redbus.in/bus-tickets/palampur-to-...


ASSAM BUS ROUTES AND LINKS

In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_AS = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_AS.maximize_window()

# Open the desired webpage
driver_AS.get("https://www.redbus.in/online-booking/astc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def assam_link_route(path):
    LINKS_AS = []
    ROUTE_AS = []
    
    # WebDriverWait instance
    wait_AS = WebDriverWait(driver_AS, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_AS.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_AS.append(d)
            
            for route in paths:
                ROUTE_AS.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_AS.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_AS.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_AS.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_AS.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_AS, ROUTE_AS

# Retrieve links and routes
LINKS_AS, ROUTE_AS = assam_link_route("//a[@class='route']")

# Close the WebDriver
driver_AS.quit()

# Save data to a CSV file
df_as = pd.DataFrame({"Route_name": ROUTE_AS, "Route_link": LINKS_AS})
df_as.to_csv("df_as.csv", index=False)


Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [20]:
df_as

,Route_name,Route_link
0,Tezpur to Guwahati,https://www.redbus.in/bus-tickets/tezpur-to-gu...
1,Guwahati to Tezpur,https://www.redbus.in/bus-tickets/guwahati-to-...
2,Nagaon (Assam) to Guwahati,https://www.redbus.in/bus-tickets/nagaon-to-gu...
3,Guwahati to Nagaon (Assam),https://www.redbus.in/bus-tickets/guwahati-to-...
4,Goalpara to Guwahati,https://www.redbus.in/bus-tickets/goalpara-to-...
5,Dhubri to Guwahati,https://www.redbus.in/bus-tickets/dhubri-to-gu...
6,Sibsagar (Assam) to North Lakhimpur,https://www.redbus.in/bus-tickets/sibsagar-to-...
7,North Lakhimpur to Sibsagar (Assam),https://www.redbus.in/bus-tickets/north-lakhim...
8,Guwahati to Dhubri,https://www.redbus.in/bus-tickets/guwahati-to-...
9,Jorhat to North Lakhimpur,https://www.redbus.in/bus-tickets/jorhat-to-no...


UTTAR PRADESH BUS ROUTES AND LINKS

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_U = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_U.maximize_window()

# Open the desired webpage
driver_U.get("https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def up_link_route(path):
    LINKS_UP = []
    ROUTE_UP = []
    
    # WebDriverWait instance
    wait_U = WebDriverWait(driver_U, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_U.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_UP.append(d)
            
            for route in paths:
                ROUTE_UP.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_U.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_U.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_U.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_U.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_UP, ROUTE_UP

# Retrieve links and routes
LINKS_UP, ROUTE_UP = up_link_route("//a[@class='route']")

# Close the WebDriver
driver_U.quit()

# Save data to a CSV file
df_up = pd.DataFrame({"Route_name": ROUTE_UP, "Route_link": LINKS_UP})
df_up.to_csv("df_up.csv", index=False)


Navigating to page 2
Navigating to page 3
Navigating to page 4
Navigating to page 5
No more pages to paginate or pagination element not found


In [22]:
df_up

,Route_name,Route_link
0,Delhi to Bareilly (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bar...
1,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...
2,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...
3,Lucknow to Gorakhpur (uttar pradesh),https://www.redbus.in/bus-tickets/lucknow-to-g...
4,Aligarh (uttar pradesh) to Delhi,https://www.redbus.in/bus-tickets/aligarh-utta...
5,Lucknow to Prayagraj(Uttar Pradesh),https://www.redbus.in/bus-tickets/lucknow-to-a...
6,Delhi to Aligarh (uttar pradesh),https://www.redbus.in/bus-tickets/delhi-to-ali...
7,Lucknow to Delhi,https://www.redbus.in/bus-tickets/lucknow-to-d...
8,Prayagraj(Uttar Pradesh) to Lucknow,https://www.redbus.in/bus-tickets/allahabad-to...
9,Delhi to Farrukhabad (Uttar Pradesh),https://www.redbus.in/bus-tickets/delhi-to-far...


WEST BENGAL BUS ROUTES AND LINKS

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_W = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_W.maximize_window()

# Open the desired webpage
driver_W.get("https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def wb_link_route(path):
    LINKS_WB = []
    ROUTE_WB = []
    
    # WebDriverWait instance
    wait_W = WebDriverWait(driver_W, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_W.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_WB.append(d)
            
            for route in paths:
                ROUTE_WB.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_W.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_W.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_W.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_W.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_WB, ROUTE_WB

# Retrieve links and routes
LINKS_WB, ROUTE_WB = wb_link_route("//a[@class='route']")

# Close the WebDriver
driver_W.quit()

# Save data to a CSV file
df_wb = pd.DataFrame({"Route_name": ROUTE_WB, "Route_link": LINKS_WB})
df_wb.to_csv("df_wb.csv", index=False)


Navigating to page 2
Navigating to page 3
Navigating to page 4
No more pages to paginate or pagination element not found


In [24]:
df_wb

,Route_name,Route_link
0,Digha to Barasat (West Bengal),https://www.redbus.in/bus-tickets/digha-to-bar...
1,Durgapur (West Bengal) to Kolkata,https://www.redbus.in/bus-tickets/durgapur-to-...
2,Digha to Kolkata,https://www.redbus.in/bus-tickets/digha-to-kol...
3,Barasat (West Bengal) to Digha,https://www.redbus.in/bus-tickets/barasat-west...
4,Kolkata to Durgapur (West Bengal),https://www.redbus.in/bus-tickets/kolkata-to-d...
5,Kolkata to Digha,https://www.redbus.in/bus-tickets/kolkata-to-d...
6,Barasat (West Bengal) to Contai (Kanthi),https://www.redbus.in/bus-tickets/barasat-west...
7,Barasat (West Bengal) to Midnapore,https://www.redbus.in/bus-tickets/barasat-west...
8,Barasat (West Bengal) to Nandakumar (west bengal),https://www.redbus.in/bus-tickets/barasat-west...
9,Barasat (West Bengal) to Kolaghat,https://www.redbus.in/bus-tickets/barasat-west...


PUNJAB BUS ROUTES AND LINKS

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementNotInteractableException
import time
import pandas as pd

# Initialize WebDriver
driver_P = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver_P.maximize_window()

# Open the desired webpage
driver_P.get("https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile")
time.sleep(3)  # Allow time for the page to load





# Function to retrieve bus links and routes
def punjab_link_route(path):
    LINKS_PB = []
    ROUTE_PB = []
    
    # WebDriverWait instance
    wait_P = WebDriverWait(driver_P, 10)
    
    # Retrieve route links from multiple pages
    while True:
        try:
            # Extract route links and texts
            paths = driver_P.find_elements(By.XPATH, path)
            for links in paths:
                d = links.get_attribute("href")
                if d:
                    LINKS_PB.append(d)
            
            for route in paths:
                ROUTE_PB.append(route.text)
            
            # Handle pagination
            try:
                # Get the active page number
                active_page_element = driver_P.find_element(By.XPATH, "//div[@class='DC_117_pageTabs DC_117_pageActive']")
                active_page_number = active_page_element.text
                next_page_number = str(int(active_page_number) + 1)  #converting the page number str to int +1 then converting to str
                
                # Locate the next page button
                next_page_button_xpath = f"//div[@class='DC_117_paginationTable']//div[text()='{next_page_number}']"
                next_page_button = wait_P.until(EC.presence_of_element_located((By.XPATH, next_page_button_xpath)))
                
                # Scroll into view and click using JavaScript if necessary
                driver_P.execute_script("arguments[0].scrollIntoView(true);", next_page_button)  #to ensure element is in view before clickin
                time.sleep(1)  # Short delay to ensure scroll action is completed
                
                try:
                    next_page_button.click()
                except ElementNotInteractableException:
                    driver_P.execute_script("arguments[0].click();", next_page_button)  #if above scrollintoview doesnt work this is used as exception for 
                                                                                     #graceful termination
                
                print(f"Navigating to page {next_page_number}")
                time.sleep(10)  # Wait for the page to load new content
            except (NoSuchElementException, TimeoutException):
                print("No more pages to paginate or pagination element not found")
                break
        except Exception as e:
            print(f"Error occurred: {str(e)}")
            break
    
    return LINKS_PB, ROUTE_PB

# Retrieve links and routes
LINKS_PB, ROUTE_PB = punjab_link_route("//a[@class='route']")

# Close the WebDriver
driver_P.quit()

# Save data to a CSV file
df_pb = pd.DataFrame({"Route_name": ROUTE_PB, "Route_link": LINKS_PB})
df_pb.to_csv("df_pb.csv", index=False)


Navigating to page 2
Navigating to page 3
No more pages to paginate or pagination element not found


In [26]:
df_pb

,Route_name,Route_link
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...
1,Patiala to Delhi,https://www.redbus.in/bus-tickets/patiala-to-d...
2,Ludhiana to Delhi,https://www.redbus.in/bus-tickets/ludhiana-to-...
3,Delhi to Ludhiana,https://www.redbus.in/bus-tickets/delhi-to-lud...
4,Ludhiana to Delhi Airport,https://www.redbus.in/bus-tickets/ludhiana-to-...
5,Delhi to Jalandhar,https://www.redbus.in/bus-tickets/delhi-to-jal...
6,Delhi Airport to Patiala,https://www.redbus.in/bus-tickets/delhi-airpor...
7,Chandigarh to Patiala,https://www.redbus.in/bus-tickets/chandigarh-t...
8,Jalandhar to Delhi,https://www.redbus.in/bus-tickets/jalandhar-to...
9,Delhi Airport to Ludhiana,https://www.redbus.in/bus-tickets/delhi-airpor...


combine all this in one dataframe

In [27]:
import pandas as pd
dfc=pd.concat([df_k,df_as,df_g,df_h,df_pb,df_r,df_s,df_t,df_up,df_wb,df_a],ignore_index=True)
dfc.to_csv('dfc.csv',index=False)

In [28]:
dfc

,Route_name,Route_link
0,Bangalore to Kozhikode,https://www.redbus.in/bus-tickets/bangalore-to...
1,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...
2,Kozhikode to Bangalore,https://www.redbus.in/bus-tickets/kozhikode-to...
3,Ernakulam to Kozhikode,https://www.redbus.in/bus-tickets/ernakulam-to...
4,Mysore to Kozhikode,https://www.redbus.in/bus-tickets/mysore-to-ko...
...,...,...
370,Tadipatri to Bangalore,https://www.redbus.in/bus-tickets/tadipatri-to...
371,Kakinada to Vijayawada,https://www.redbus.in/bus-tickets/kakinada-to-...
372,Visakhapatnam to Amalapuram,https://www.redbus.in/bus-tickets/visakhapatna...
373,Hyderabad to Markapuram,https://www.redbus.in/bus-tickets/hyderabad-to...
